In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from collections import defaultdict

from plato.planets.metrics import PlanetPopulationMetrics
from plato.planets.populations import PopulationModel
from plato.stars import filter_p1_targets, filter_valid_targets
from plato.visualisation import (
    FigureProcessor,
    contour_plot,
    get_earth_marker,
    get_palette,
    set_plot_defaults,
)

In [3]:
# set_plot_defaults()

figure_directory = f"../figures/06_planet_populations/"
save = True

In [4]:
LOPS2 = pd.read_csv(f"../data/processed/LOPS2_targets.csv")
LOPN1 = pd.read_csv(f"../data/processed/LOPN1_targets.csv")

fields = pd.concat([LOPS2, LOPN1])
fields = filter_valid_targets(fields)
fields = fields[
    [
        "Radius",
        "Mass",
        "[Fe/H]",
        "u1",
        "u2",
        "gaiaV",
        "n_cameras",
        "Population",
    ]
]

fields["cos_i"] = 0
fields["sigma_star"] = 10e-6
fields = fields.rename(
    columns={
        "Radius": "R_star",
        "Mass": "M_star",
        "gaiaV": "Magnitude_V",
    }
)

p1_sample = filter_p1_targets(fields)

In [5]:
metrics = PlanetPopulationMetrics()

## Recreate NGPPS II Table 7

In [ ]:
metric_table = {}

for num_embryos in [10, 20, 50, 100]:
    population_model = PopulationModel(
        fields,
        num_embryos=num_embryos,
        additional_columns=["Population"],
    )

    mock = population_model.create_mock_population(additional_columns=["Population"])

    table = metrics.calculate_metrics(
        mock,
        by_population=True,
        number_of_systems={
            pop: len(fields[fields["Population"] == pop])
            for pop in fields["Population"].unique()
        },
    )
    metric_table[num_embryos] = table

metric_table = pd.concat(metric_table, axis=1)

In [ ]:
metric_table.reindex(
    index=pd.MultiIndex.from_product(
        [
            [
                "Thin Disk",
                "Thick Disk Candidate",
                "Thick Disk",
                "Halo Candidate",
                "Halo",
            ],
            ["Earth", "Super-Earth", "Neptunian", "Sub-Giant", "Giant"],
        ],
    )
)

## GET UNCERTAINTIES FOR HALO PLANETS, ALSO METALLICITY CUT

In [45]:
halo_table = defaultdict(dict)

for num_embryos in [10, 20, 50, 100]:
    for metallicity_name, metallicity_cut in {
        "Without Limit": None,
        "With Limit": -0.6,
    }.items():
        halo_population_model = PopulationModel(
            fields[fields["Population"] == "Halo"],
            num_embryos=num_embryos,
            additional_columns=["Population"],
        )

        halo_mocks = [
            halo_population_model.create_mock_population(
                additional_columns=["Population"],
                metallicity_limit=metallicity_cut,
            )
            for _ in range(1000)
        ]

        halo_metrics = [
            metrics.calculate_number_of_planets(
                mock,
            )
            for mock in halo_mocks
        ]

        halo_metrics_stats = metrics.calculate_metrics_stats(
            halo_metrics,
            grouping_column="Planet Category",
            format_uncertainties=True,
            round_decimals="ceil",
            to_int=True,
        )

        halo_table[num_embryos][metallicity_name] = halo_metrics_stats

halo_table = pd.concat(
    {key: pd.concat(val, axis=0) for key, val in halo_table.items()},
    axis=1,
)
halo_table.columns = halo_table.columns.droplevel(1)

In [46]:
halo_table.reindex(["Earth", "Super-Earth", "Neptunian", "Sub-Giant", "Giant"], level=1)

10   \
              Planet Category                             
Without Limit Earth               $465.0^{+2.0}_{-5.0}$   
              Super-Earth      $1338.0^{+15.0}_{-19.0}$   
              Neptunian            $82.0^{+9.0}_{-3.0}$   
              Sub-Giant           $177.0^{+7.0}_{-7.0}$   
              Giant               $34.0^{+2.0}_{-11.0}$   
With Limit    Earth               $77.0^{+10.0}_{-5.0}$   
              Super-Earth         $143.0^{+5.0}_{-2.0}$   
              Neptunian            $29.0^{+3.0}_{-6.0}$   
              Sub-Giant            $18.0^{+3.0}_{-5.0}$   
              Giant                $15.0^{+3.0}_{-3.0}$   

                                                    20   \
              Planet Category                             
Without Limit Earth             $916.0^{+37.0}_{-13.0}$   
              Super-Earth      $1334.0^{+57.0}_{-15.0}$   
              Neptunian          $200.0^{+11.0}_{-5.0}$   
              Sub-Giant            $53.0^{+9.0}_{-9.0}$   
              Giant                $38.0^{+1.0}_{-7.0}$   
With Limit    Earth               $178.0^{+3.0}_{-9.0}$   
              Super-Earth        $203.0^{+1.0}_{-10.0}$   
              Neptunian            $38.0^{+3.0}_{-7.0}$   
              Sub-Giant             $9.0^{+5.0}_{-1.0}$   
              Giant                $16.0^{+0.0}_{-3.0}$   

                                                    50   \
              Planet Category                             
Without Limit Earth            $1915.0^{+49.0}_{-21.0}$   
              Super-Earth        $434.0^{+3.0}_{-29.0}$   
              Neptunian            $26.0^{+9.0}_{-3.0}$   
              Sub-Giant             $5.0^{+0.0}_{-2.0}$   
              Giant                $24.0^{+2.0}_{-2.0}$   
With Limit    Earth              $278.0^{+31.0}_{-2.0}$   
              Super-Earth         $227.0^{+0.0}_{-9.0}$   
              Neptunian           $30.0^{+11.0}_{-3.0}$   
              Sub-Giant             $4.0^{+0.0}_{-1.0}$   
              Giant                $17.0^{+3.0}_{-3.0}$   

                                                    100  
              Planet Category                            
Without Limit Earth            $2941.0^{+52.0}_{-59.0}$  
              Super-Earth        $531.0^{+49.0}_{-1.0}$  
              Neptunian            $22.0^{+3.0}_{-7.0}$  
              Sub-Giant            $13.0^{+6.0}_{-2.0}$  
              Giant                $28.0^{+3.0}_{-4.0}$  
With Limit    Earth               $413.0^{+7.0}_{-3.0}$  
              Super-Earth       $262.0^{+15.0}_{-15.0}$  
              Neptunian            $17.0^{+6.0}_{-7.0}$  
              Sub-Giant            $18.0^{+4.0}_{-7.0}$  
              Giant                $24.0^{+4.0}_{-1.0}$

## OCCURENCE RATE PLOT